In [ ]:
import os
os.environ['HTTP_PROXY'] = "http://127.0.0.1:10809"
os.environ['HTTPS_PROXY'] = "http://127.0.0.1:10809"

In [ ]:
import geemap
import ee
Map=geemap.Map()
Map

In [ ]:
chongqing = ee.FeatureCollection('users/311605001111/YangtzeCity/chongqing')
Map.addLayer(chongqing, {}, "chongqing")
Map.centerObject(chongqing,7)

In [ ]:
def NDVI(image):
    ndvi=image.normalizedDifference(['B5', 'B4']).rename('NDVI')  
    return image.addBands(ndvi)

def mNDWI(image):
    mndwi=image.normalizedDifference(['B3', 'B6']).rename("mNDWI")
    return image.addBands(mndwi)

def EVI(image):
    evi = image.expression('((nir - red) / (nir + 6*red - 7.5*blue + 1)) * 2.5',
        {
          'blue': image.select('B2'),    # 0.452-0.512μm, blue
          'nir': image.select('B5'),    # 0.851-0.879μm, nir
          'red': image.select('B4'),    # 0.636-0.673μm, red
        }).rename('EVI')
    return image.addBands(evi)
# AWEI(自动水体提取指数): AWEIsh = B2 + 2.5B3 - 1.5(B5+B6) - 0.25B7
def AWEIsh(image):
    awei = image.expression('B2 + 2.5*B3 - 1.5*(B5+B6) - 0.25*B7',
        {
          'B2': image.select('B2'),
          'B3': image.select('B3'),    
          'B5': image.select('B5'),    
          'B6': image.select('B6'),
          'B7': image.select('B7'),
        }).rename('AWEI')
    return image.addBands(awei)

In [ ]:
def area_after_clip(image):
    image_clip = image.clip(chongqing).select('B2').gt(0)
    image_area = image_clip.multiply(ee.Image.pixelArea()).divide(1e6)
    areas = image_area.reduceRegion(**{
        'reducer': ee.Reducer.sum(),
        'geometry': chongqing.geometry(),
        'scale': 30,
        'maxPixels': 1e14
    })
    return image.clip(chongqing).set({'area': areas.get('B2')})

In [ ]:
# 去云掩膜函数
def maskL8sr(image):
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    qa = image.select('pixel_qa')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                 .And(qa.bitwiseAnd(cloudsBitMask).eq(0))
    return image.updateMask(mask)

# 可视化参数设置
vis_params = {
  'bands': ['B4', 'B3', 'B2'],
  'min': 0,
  'max': 3000,
  'gamma': 1.4,
}

In [ ]:
# 筛选数据（使用日期，云量，研究区），去云
landsat_images = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filterBounds(chongqing) \
    .filterDate('2019-01-01', '2019-12-31') \
    .map(maskL8sr)


In [ ]:
print(landsat_images.size().getInfo())

In [ ]:
landsat_images_filter = landsat_images.map(area_after_clip).filter(ee.Filter.gt('area',0))
Map.addLayer(landsat_images_filter.first(),vis_params,'chongqing landsat')
print(landsat_images_filter.size().getInfo())

In [ ]:
landsat_map = landsat_images_filter.map(NDVI).map(mNDWI).map(EVI).map(AWEIsh)
chongqing_landsat = landsat_map.mosaic().clip(chongqing)
Map.addLayer(chongqing_landsat,vis_params,'chongqing landsat')

In [ ]:
elevation = ee.Image('CGIAR/SRTM90_V4').select('elevation')
slope = ee.Terrain.slope(elevation)
chongqing_slope = slope.clip(chongqing)
# Map.addLayer(chongqing_slope,{min: 0, max: 60}, 'chongqing slope')


flat_slope = slope.clip(chongqing).select('slope').lt(3).selfMask()
Map.addLayer(flat_slope,{'palette': 'yellow'}, 'flat slope')

In [ ]:
water_mndwi = chongqing_landsat.select('mNDWI').gt(0.15)
water_ndvi_mndwi = chongqing_landsat.select('mNDWI').gt(chongqing_landsat.select('NDVI'))
water_awei = chongqing_landsat.select('AWEI').gt(0)

water_mndwi_rename = water_mndwi.select('mNDWI').rename('class')
water_ndvi_mndwi_rename = water_ndvi_mndwi.select('mNDWI').rename('class')
water_awei_rename = water_awei.select('AWEI').rename('class')
chongqing_water = ee.ImageCollection([water_mndwi_rename,water_ndvi_mndwi_rename,water_awei_rename]).sum().eq(3).selfMask()
Map.addLayer(chongqing_water,{'palette': 'blue'},'chonging water')

In [ ]:
chongqing_water_slope = chongqing_water.updateMask(flat_slope)
Map.addLayer(chongqing_water_slope,{'palette': 'cyan'},'chonging water slope')

In [ ]:
water_data = ee.Image("JRC/GSW1_2/YearlyHistory/2019").clip(chongqing)
water = water_data.eq(3).selfMask()
Map.addLayer(water,{'palette': '99d9ea'}, 'water')

In [ ]:
first = area_after_clip(landsat_images.first())
print(first.get('area').getInfo())
print(first.getInfo())

In [ ]:
fir = landsat_images.first().clip(chongqing)
first = landsat_images.first().clip(chongqing).select('B2').gt(0)
Map.addLayer(first,{'palette': 'blue'},'first landsat')
print(first.getNumber('system:time_start').getInfo())
image_area = first.multiply(ee.Image.pixelArea()).divide(1e6)
areas = image_area.reduceRegion(**{
    'reducer': ee.Reducer.sum(),
    'geometry': chongqing.geometry(),
    'scale': 30,
    'maxPixels': 1e14
})

b = fir.set({'area': areas.get('B2')})
print(b.get('area').getInfo())
print(areas.getNumber('B2').getInfo())